### Deep Learning Up and Running
#### By MiMoTrix
##### MNIST with FullyConnected Network

In [19]:
import tensorflow as tf
import time


In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
learning_rate = 0.01
epochs = 30
batch_size = 100

##### Weight Variables Initialize

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

##### Bias Variables Initialize

In [20]:
def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

##### Inputs

In [ ]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

KeepProbInput = tf.placeholder(tf.float32) # Against Biasing
X_Drop = tf.nn.dropout(X, keep_prob=KeepProbInput)
Weights_HL_One = weight_variable([784, 500])
Biases_HL_One = bias_variable([500])
Output_HL_One = tf.nn.relu(tf.matmul(X_Drop, Weights_HL_One) + Biases_HL_One)

KeepProbInput = tf.placeholder(tf.float32) # Against Biasing
Output_Drop_LayerOne = tf.nn.dropout(Output_HL_One, keep_prob=KeepProbInput)
Weights_HL_Two = weight_variable([500, 500])
Biases_HL_Two = bias_variable([500])
Output_HL_Two = tf.nn.relu(tf.matmul(Output_Drop_LayerOne, Weights_HL_Two) + Biases_HL_Two)

Output_Drop_LayerTwo = tf.nn.dropout(Output_HL_Two, KeepProbInput)
Weights_HL_Output = weight_variable([500, 10])
Biases_HL_Output = bias_variable([10])
Output = tf.nn.softmax(tf.matmul(Output_Drop_LayerTwo, Weights_HL_Output) + Biases_HL_Output)

NameError: name 'W_fc3' is not defined